In [3]:
import pandas as pd
import requests
import json
import os
from datetime import datetime, timedelta

# Fetch Streamflow Data

In [ ]:
if os.path.exists('yellowstone6192500.csv'):
    df_existing = pd.read_csv('yellowstone6192500.csv')
    most_recent_date = df_existing['datetime'].max()
else:
    most_recent_date = None


2024-12-23 11:45


/tmp/ipykernel_139539/1224773202.py:2: DtypeWarning: Columns (4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_existing = pd.read_csv('yellowstone6192500.csv')


In [12]:
# example url format: https://waterservices.usgs.gov/nwis/iv/?format=rdb&sites=06192500&startDT=2024-12-22T04:10&endDT=2025-01-18T04:10&siteStatus=all

# get the most recent data for 06192500
usgs_site = '06192500'
most_recent_datetime = datetime.strptime(most_recent_date, '%Y-%m-%d %H:%M')
start_date = (most_recent_datetime - timedelta(hours=1)).strftime('%Y-%m-%dT%H')
end_date = datetime.now().strftime('%Y-%m-%dT%H')
url = f'https://waterservices.usgs.gov/nwis/iv/?format=rdb&sites={usgs_site}&startDT={start_date}&endDT={end_date}&siteStatus=all'
response = requests.get(url)
data = response.text

# print(data)

# parse the data
# There  are 31  lines  of  metadata  at  the  beginning  of  the  file.  The  data  starts  on  line  32.  
# The  data  is  tab-delimited  and the initial rows can  be deleted
data = data.split('\n')[32:]
data = [line.split('\t') for line in data]
df = pd.DataFrame(data, columns=['agency_cd', 'id', 'datetime', 'timezone', 'temp', 'temp_quality', 'discharge', 'discharge_quality', 'gage_height', 'gage_height_quality'])
df.drop(columns=['temp', 'temp_quality', 'discharge_quality', 'gage_height', 'gage_height_quality'], inplace=True)
print(df)

     agency_cd        id          datetime timezone discharge
0         USGS  06192500  2024-12-23 10:00      MST      1210
1         USGS  06192500  2024-12-23 10:15      MST      1200
2         USGS  06192500  2024-12-23 10:30      MST      1210
3         USGS  06192500  2024-12-23 10:45      MST      1210
4         USGS  06192500  2024-12-23 11:00      MST      1200
...        ...       ...               ...      ...       ...
7858      USGS  06192500  2025-03-17 03:15      MDT      1160
7859      USGS  06192500  2025-03-17 03:30      MDT      1170
7860      USGS  06192500  2025-03-17 03:45      MDT      1160
7861      USGS  06192500  2025-03-17 04:00      MDT      1180
7862                None              None     None      None

[7863 rows x 5 columns]


In [ ]:
df_csv = pd.read_csv('yellowstone6192500.csv')
columns_to_drop = ['temp', 'temp_quality', 'discharge_quality', 'gage_height', 'gage_height_quality']
df_csv.drop(columns=[col for col in columns_to_drop if col in df_csv.columns], inplace=True)

if list(df.columns) == list(df_csv.columns):
    print("Headers are the same.")
else:
    print("Headers are different.")

df_csv = pd.concat([df_csv, df], ignore_index=True)

df_csv.drop_duplicates(subset=['datetime'], keep='last', inplace=True)

save_path = 'yellowstone6192500.csv'
if list(df.columns) == list(df_csv.columns):
    df_csv.to_csv(save_path, index=False)
else:
    print("Stopped as headers are different.")
print(f"Data saved to {save_path}")

/tmp/ipykernel_139539/4279589922.py:1: DtypeWarning: Columns (4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv('yellowstone6192500.csv')


Headers are the same.
Data saved to yellowstone6192500.csv


# Fetch Snow Data